In [77]:
from pymongo import MongoClient
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import re

In [54]:
client = MongoClient()
db = client.twitter
coll = db.tweets

In [58]:
# for tweet in coll.find().limit(1):
#     print tweet['text']

@politico: He's condoning mass suicide for Repubs. I knew there was something noble about this man.


In [59]:
tweets_raw = [''.join(tweet['text']).lower() for tweet in coll.find()]

In [62]:
print len(tweets_raw)

327


In [63]:
# Create a set of tokenized tweets
tweets_tokenized = [word_tokenize(tweet) for tweet in tweets_raw]

In [79]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
    ]

In [64]:
print tweets_tokenized

[[u'@', u'politico', u':', u'he', u"'s", u'condoning', u'mass', u'suicide', u'for', u'repubs', u'.', u'i', u'knew', u'there', u'was', u'something', u'noble', u'about', u'this', u'man', u'.'], [u'security', u'tightened', u'in', u'kumbakonam', u':', u'suicide', u'bombers', u'on', u'the', u'prowl', u',', u'claims', u'an', u'anonymous', u'letter', u'.', u'https', u':', u'//t.co/io6gbnr7zo'], [u'rt', u'@', u'ombtruebelieber', u':', u'suicide', u'hotline', u'on', u'the', u'screen', u'you', u'guys', u'are', u'more', u'than', u'investigators', u'touching', u'real', u'issues', u'on', u'the', u'show', u'is', u'great', u'@', u'zak_baga\u2026'], [u'rt', u'@', u'ombtruebelieber', u':', u'suicide', u'hotline', u'on', u'the', u'screen', u'you', u'guys', u'are', u'more', u'than', u'investigators', u'touching', u'real', u'issues', u'on', u'the', u'show', u'is', u'great', u'@', u'zak_baga\u2026'], [u'rt', u'@', u'emiforlove', u':', u'that', u"'s", u'so', u'very', u'sad', u'.', u'our', u'teen', u'suicide

In [70]:
wordnet = WordNetLemmatizer()
tweets = [[wordnet.lemmatize(word) for word in words] for words in tweets_tokenized]

In [71]:
print tweets

[[u'@', u'politico', u':', u'he', u"'s", u'condoning', u'mass', u'suicide', u'for', u'repubs', u'.', u'i', u'knew', u'there', u'wa', u'something', u'noble', u'about', u'this', u'man', u'.'], [u'security', u'tightened', u'in', u'kumbakonam', u':', u'suicide', u'bomber', u'on', u'the', u'prowl', u',', u'claim', u'an', u'anonymous', u'letter', u'.', u'http', u':', u'//t.co/io6gbnr7zo'], [u'rt', u'@', u'ombtruebelieber', u':', u'suicide', u'hotline', u'on', u'the', u'screen', u'you', u'guy', u'are', u'more', u'than', u'investigator', u'touching', u'real', u'issue', u'on', u'the', u'show', u'is', u'great', u'@', u'zak_baga\u2026'], [u'rt', u'@', u'ombtruebelieber', u':', u'suicide', u'hotline', u'on', u'the', u'screen', u'you', u'guy', u'are', u'more', u'than', u'investigator', u'touching', u'real', u'issue', u'on', u'the', u'show', u'is', u'great', u'@', u'zak_baga\u2026'], [u'rt', u'@', u'emiforlove', u':', u'that', u"'s", u'so', u'very', u'sad', u'.', u'our', u'teen', u'suicide', u'rate'

In [76]:
y = np.random.choice([0, 1], size=(len(tweets),))
print y

[1 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 0
 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 1 0 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 0
 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1
 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1
 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 0 1
 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 0 1]


In [73]:
len(tweets)

327

In [72]:
##
tfidf = TfidfVectorizer(stop_words='english')
tfidfed = tfidf.fit_transform(tweets)

AttributeError: 'list' object has no attribute 'lower'

In [44]:
pwd

u'/Users/datascientist/Desktop/YZ'